# Image classification using tensorflow pre-trained model

Tensorflow-Slim image classification model library provides both the implementation and pre-trianed checkpoint many popular convolution nerual nets for image classification.

Using TFNet in Analytics-Zoo, we can easily load these pre-trained model and make distributed inference with only a few lines of code.

## Add the slim image classification model library to $PYTHONPATH

In [1]:
import sys
import tensorflow as tf
sys.path
slim_path = "/tmp/tensorflow/models/research/slim" # Please set this to the directory where you clone the tensorflow models repository
sys.path.append(slim_path)

## Construct the inference graph and restore the checkpoint

In [2]:
from datasets import dataset_factory
from nets import inception
slim = tf.contrib.slim

image = tf.random_uniform(shape=[224, 224, 3])
batch_size_tensor = tf.placeholder_with_default(32, shape=())
images = tf.train.batch(
    [image],
    batch_size=batch_size_tensor,
    num_threads=4,
    capacity=5 * 32)
# One thing to note is that the batch_size must be placeholder, so that batch size is not hard coded into graph and analytics-zoo can work on different batch sizes.
# The above code can be replaced with a single line of code
# images = tf.placeholder(dtype=tf.float32, shape=(None, 224, 224, 3))

with slim.arg_scope(inception.inception_v1_arg_scope()):
    logits, end_points = inception.inception_v1(images, num_classes=1001, is_training=False)

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, "/tmp/models/inception_v1.ckpt") # You need to edit this path to the checkpoint you downloaded

## Export the graph as a frozen inference graph

The export_tf utility function will frozen the tensorflow graph, strip unused operation according to the inputs and outputs and save it to the specified directory along with the input/output tensor names.

In [3]:
from zoo.util.tf import export_tf
export_tf(sess, "/tmp/models/tfnet", inputs=[images], outputs=[logits])

Converted 230 variables to const ops.


## Load to Analytics-Zoo

In [4]:
from zoo.pipeline.api.net import TFNet
model = TFNet.from_export_folder("/tmp/models/tfnet")

creating: createTFNet


## Test it on one image

![Test Image](./test.jpg)

In [5]:
import cv2
import numpy as np
import json
im = cv2.imread("test.jpg")
im = cv2.resize(im, (224, 224))
im = (im - 127.0) / 128.0
im = np.expand_dims(im, 0)

In [6]:
import json
with open("imagenet_class_index.json") as f:
    class_idx = json.load(f)
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In [7]:
import numpy as np
result = model.predict([im])
print(idx2label[np.argmax(result, 1)[0]])

Persian_cat
